In [1]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import month,year,col,sum
from snowflake.snowpark.version import VERSION
# Misc
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

In [4]:
param = json.load(open('connection_movie.json'))

In [7]:
session = Session.builder.configs(param).create()

In [8]:
session

In [10]:
session.sql_simplifier_enabled = True

In [11]:
snowflake_environment = session.sql('select current_user(), current_version()').collect()

In [13]:
type(snowflake_environment)

list

In [14]:
snowflake_environment

[Row(CURRENT_USER()='NADINA', CURRENT_VERSION()='7.17.0')]

In [15]:
session.get_current_role()

'"ACCOUNTADMIN"'

In [17]:
session.get_current_database(), session.get_current_schema(), session.get_current_warehouse()

('"MOVIE_DB"', '"MOVIE_SCHEMA"', '"MOVIE_L"')

In [28]:
tables = session.sql('show tables').collect()

In [39]:
tables[0]['name']

'ITEMS'

In [40]:
for table in tables:
    print(table['name'])

ITEMS
RATINGS
USERS
